## IMPORTING MODULES

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score
import re
from sklearn.linear_model import LogisticRegression
# Binary Relevance
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

## Reading Data
### Metadata representing training set
### test_data representing testing set

In [3]:
data = pd.read_csv("Metadata.csv")

In [4]:
test_data = pd.read_csv("test_data.csv")

In [5]:
data

,Name,Year,Rating,Metascore,Directors,Cast,Genre,Description,Gross,Duration,...,Editorial Dept,Production Designers,Art Directors,Costume Designers,Production Manager,Assistant Director,Sound Department,Camera Department,Music Department,Production Companies
0,Godfather,1972,9.2,100.0,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...","['Crime', 'Drama']","The Godfather ""Don"" Vito Corleone is the head ...","{'Budget': '$6,000,000 (estimated)', 'Opening ...",['175'],...,"Christopher Gillaspie, Kathleen Largay, Walter...",Dean Tavoularis,Warren Clymer,Anna Hill Johnstone,"Fred C. Caruso, Valerio De Paolis, Ned Kopp","Paul Baxley, Tony Brandt, Fred T. Gallo, Steph...","Charles Grenzbach, Christopher Newman, Richard...","Michael Chapman, Bill Butler, Caleb Deschanel,...","Carlo Savina, Carl Fortina, John C. Hammell, T...","Paramount Pictures, Alfran Productions"
1,Shawshank Redemption,1994,9.3,80.0,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",['Drama'],Chronicles the experiences of a formerly succe...,"{'Budget': '$25,000,000 (estimated)', 'Opening...",['142'],...,"Jeff Canavan, Patricia A. Galvin, David Leslie...","Terence Marsh, Soheil",Peter Landsdown Smith,Elizabeth McBride,"Kokayi Ampah, David V. Lester, Sue Bea Montgomery","Michael Greenwood, Thomas Schellenberg, John R...","Petra Bach, Kevin Bartnof, David Behle, Bruce ...","Tim L. Amstutz, John Archibald, Robin Brown, R...","Bill Bernstein, Julian Bratolyubov, Jim Makiej...",Castle Rock Entertainment
2,Schindler's List,1993,8.9,94.0,Steven Spielberg,"Liam Neeson, Ben Kingsley, Ralph Fiennes, Caro...","['Biography', 'Drama', 'History']",Oskar Schindler is a vain and greedy German bu...,"{'Budget': '$22,000,000 (estimated)', 'Opening...",['195'],...,"Gary Burritt, Dale Caldwell, Alan Cody, Martin...",Allan Starski,NaN,Anna B. Sheppard,"Martin Cohen, Józef Jarosz, Branko Lustig, Zdr...","Marek Brodzki, Michael Helfand, Sergio Mimica-...","Kevin Bartnof, Phil Brewster, Charles L. Campb...","Marek Bojsza, Scott Buckler, Mauro Fiore, Jare...","Andy Bass, Tom Brown, Sandy DeCrescent, Greg D...","Universal Pictures, Amblin Entertainment"
3,Raging Bull,1980,8.2,89.0,Martin Scorsese,"Robert De Niro, Cathy Moriarty, Joe Pesci, Fra...","['Biography', 'Drama', 'Sport']",When Jake LaMotta steps into a boxing ring and...,"{'Budget': '$18,000,000 (estimated)', 'Opening...",['129'],...,"Craig Bassett, Donah Bassett, Melissa Brethert...",NaN,NaN,"John Boxer, Richard Bruno",James D. Brubaker,"Henry Bronchtein, Elie Cohn, Jerry Grandey, Jo...","Michael Evje, Gary S. Gerlich, Walter A. Gest,...","Dustin Blauvelt, Peter J. Breen, Robert Connor...","Jim Henrikson, Orchestra del Teatro Comunale d...",Chartoff-Winkler Productions
4,Casablanca,1942,8.5,100.0,Michael Curtiz,"Humphrey Bogart, Ingrid Bergman, Paul Henreid,...","['Drama', 'Romance', 'War']","The story of Rick Blaine, a cynical world-wear...","{'Budget': '$950,000 (estimated)', 'Opening We...",['102'],...,Don Siegel,NaN,Carl Jules Weyl,Orry-Kelly,Al Alleborn,Lee Katz,"Francis J. Scheid, Edward Ullman","Mike Joyce, Wally Meinardus","Leo F. Forbstein, Hugo Friedhofer, M.K. Jerome...",Warner Bros.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Rear Window,1954,8.4,100.0,Alfred Hitchcock,"James Stewart, Grace Kelly, Wendell Corey, The...","['Mystery', 'Thriller']","Professional photographer L.B. ""Jeff"" Jefferie...","{'Budget': '$1,000,000 (estimated)', 'Opening ...",['112'],...,NaN,NaN,"J. McMillan Johnson, Hal Pereira",Edith Head,C.O. Erickson,"Herbert Coleman, Lloyd Allen","John Cope, Harry Lindgren, Howard Beals, Loren...","Bud Fraker, Vic Jones, William Schurr, Leonard...","Sidney Cutner, Gus Levene, Leonid Raab, Leo Sh...",Alfred J. Hitchcock Productions
96,Third Man,1949,8.1,97.0,Carol Reed,"Joseph Cotten, Alida Valli, Orson Welles, Trev...","['Film-Noir', 'Mystery', 'Thriller']","An out of work pulp fiction novelist, Holly Ma...","{'Opening We

In [6]:
test_data

,Name,Genre
0,Avatar,"Action', 'Adventure', 'Fantasy'"
1,Bad Boys,"Action', 'Comedy', 'Crime'"
2,Aliens Vs Predator,"Action', 'Adventure', 'Horror'"
3,The Avengers,"Action', 'Adventure', 'Sci-Fi '"
4,Hellboy,"Action', 'Fantasy', 'Horror'"
5,Hudson Hawk,"Action', 'Adventure','Comedy'"
6,Aladdin,"Adventure','Family','Fantasy'"
7,Annie Hall,"Comedy','Romance'"
8,Three Kings,"Action','Adventure','Comedy'"
9,Joker,"Crime','Drama','Thriller'"


In [7]:
text_data=[]
genre_data=[]
failed_movies = []
for i in data.Name:
    try:
        f = open("scripts/"+i+ ".txt")
        text_data.append(f.read())
        temp=[]
        for j in data[data['Name']==i]['Genre'].values[0][1:-1].replace("'","").split(','):
            temp.append(j.strip())
        genre_data.append(temp)
    except:
        failed_movies.append(i)
        continue

In [8]:
text_data_test=[]
genre_data_test=[]
#failed_movies = []
for i in test_data.Name:
    try:
        f = open("test_scripts/"+i+ ".txt")
        text_data_test.append(f.read())
        temp=[]
        for j in test_data[test_data['Name']==i]['Genre'].values[0][0:-1].replace("'","").split(','):
            temp.append(j.strip())
        genre_data_test.append(temp)
    except:
        continue

In [9]:
genre_data_test

[['Action', 'Adventure', 'Fantasy'],
 ['Action', 'Comedy', 'Crime'],
 ['Action', 'Adventure', 'Horror'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Action', 'Fantasy', 'Horror'],
 ['Action', 'Adventure', 'Comedy'],
 ['Adventure', 'Family', 'Fantasy'],
 ['Comedy', 'Romance'],
 ['Action', 'Adventure', 'Comedy'],
 ['Crime', 'Drama', 'Thriller'],
 ['Horror'],
 ['Crime', 'Horror', 'Thriller'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Action', 'Drama', 'Music'],
 ['Comedy', 'Mystery', 'Romance'],
 ['Comedy', 'Drama', 'Romance'],
 ['Action', 'Drama', 'Thriller'],
 ['Action', 'Crime', 'Fantasy'],
 ['Crime', 'Drama', 'Mystery'],
 ['Action', 'Adventure', 'Comedy']]

# Pre-Processing Text Data

In [10]:
def clean_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    
    return text

In [11]:
new_text=[]
for text in text_data:
    new_text.append(clean_text(text))

In [12]:
new_text_test=[]
for text in text_data_test:
    new_text_test.append(clean_text(text))

In [13]:
def tokenize(text):
    '''
    Tokenize and clean text
    Input:
        text: original message text
    Output:
        lemmed: Tokenized, cleaned, and lemmatized text
    '''
    words = word_tokenize(text)
    # Remove Stopwords
    words = [w for w in words if w not in stopwords.words('english')]
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    lemmed = [lemmatizer.lemmatize(w, pos='n').strip() for w in words]
    lemmed = [lemmatizer.lemmatize(w, pos='v').strip() for w in lemmed]
    
    return lemmed


# Creaing New DataFrames for Training and Testing 

In [14]:
train=pd.DataFrame({"text":new_text,'genre':genre_data})

In [15]:
test=pd.DataFrame({"text":new_text_test,'genre':genre_data_test})

In [16]:
train.head()


,text,genre
0,the godfather screenplay by mario puzo and fra...,"[Crime, Drama]"
1,the shawshank redemption by frank darabont bas...,[Drama]
2,schindlers list by steven zaillian final draft...,"[Biography, Drama, History]"
3,music in stone cold dead in the market by loui...,"[Biography, Drama, Sport]"
4,fade in insert a revolving globe when it stops...,"[Drama, Romance, War]"


In [17]:
test.head()

,text,genre
0,avatar written by james cameron the sound of d...,"[Action, Adventure, Fantasy]"
1,bad boys revised draft by doug richardson prop...,"[Action, Comedy, Crime]"
2,aliens vs predator dark horse prod presents al...,"[Action, Adventure, Horror]"
3,the avengers written by joss whedon and there ...,"[Action, Adventure, Sci-Fi]"
4,hellboy by guillermo del toro production draft...,"[Action, Fantasy, Horror]"


In [18]:
X= train['text'].values

In [ ]:
train['tokens'] = train['text'].apply(lambda x : tokenize(x))
test['tokens'] = test['text'].apply(lambda x : tokenize(x))

In [ ]:
train.head()

In [ ]:
test.head()

# To remove the imbalance in training data, removing drama where drama is present along with other genres

In [ ]:
for row in train['genre']:
    if len(row)>1:
        if "Drama" in row:
            row.remove("Drama")

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(train['genre'])
Y=multilabel_binarizer.transform(train['genre'])

In [ ]:
count_vector_list_train=[]
for i in train['tokens']:
    count_vector_list_train.append(" ".join(i))
train['token_str']=count_vector_list_train

In [ ]:
count_vector_list_test=[]
for i in test['tokens']:
    count_vector_list_test.append(" ".join(i))
test['token_str']=count_vector_list_test

# Model
###  Using tfidf Vectoriser to encode the text data and further using Random Forest with OneVsRest Classifier for Multilabel classification to predict multiple genres

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)

In [ ]:
xtrain_tfidf = tfidf_vectorizer.fit_transform(train['token_str'])
xtest_tfidf = tfidf_vectorizer.transform(test['token_str'])

In [ ]:
# lr = LogisticRegression()
rf = RandomForestClassifier(max_depth =30,min_samples_leaf=5,min_samples_split=5,n_estimators=100)
clf = OneVsRestClassifier(rf)

clf.fit(xtrain_tfidf, Y)



In [ ]:
predicted_values_test = clf.predict_proba(xtest_tfidf)

In [ ]:
Y_test=multilabel_binarizer.transform(test['genre'])

# Setting Threshold to 0.23

In [ ]:
from sklearn.metrics import f1_score
def threshold_predict(pred,t):
    y_pr = (pred >= t).astype(int)
    return y_pr
predicted_binary_test = threshold_predict(predicted_values_test,.22)

# Finding f1 score and making confusion matrix

In [ ]:
f1score_test = f1_score(Y_test,predicted_binary_test, average="micro")

In [ ]:
print("F1 Score: "+ str(f1score_test))

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import sklearn

In [ ]:
sklearn.metrics.multilabel_confusion_matrix(Y_test,predicted_binary_test,labels=None)

In [ ]:
result = multilabel_binarizer.inverse_transform(predicted_binary_test)

In [ ]:
file = test_data[['Name','Genre']]
file['predicted_genre_multiple'] = result


In [ ]:
submission.to_csv("Predicted_Genre.csv")